In [10]:
import os, sys
# import openai
from openai import AzureOpenAI
# !pip install python-dotenv
from dotenv import load_dotenv
import json

In [ ]:
def APIKeyManager(model_type, key_path):
    
    load_dotenv(dotenv_path=key_path, override=True)
    if model_type=='azure':
        client = AzureOpenAI(
            api_version=os.environ["AZURE_API_VERSION"],
            azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
            api_key=os.environ["AZURE_API_KEY"],
        )
        return client
    elif model_type=='fanar':
        pass
    elif model_type=='gemini':
        pass
    return client

# Load environment variables
model_type="azure"
deployment = APIKeyManager(model_type, "./azure.env")


In [81]:
class PodcastOutlineGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model

    def generate_outline(self, topic, information,host_persona, guest_persona,):
        prompt = f"""
You are tasked with creating a clear and engaging structured outline for a podcast episode on the topic: '{topic}'.

Use the following personas clearly:
- Host: '{host_persona}'
- Guest: '{guest_persona}'

Base your outline on this provided information:
{information}

Organize your outline strictly in the following JSON format, using these exact keys:

{{
    "Intro1": {{
        "description": "Brief overview of the podcast and general welcome from the host.",
        "script": ["Engaging and enthusiastic introduction by the host."]
    }},
    "Intro2": {{
        "description": "Introduction of today's topic and brief bio of the guest speaker.",
        "script": ["Introduction to topic by host.", "Warm welcome and brief guest introduction."]
    }},
    "Points": {{
        "talking_points": {{
            "Point Title 1": {{
                "discussion": "Brief discussion of talking point.",
                "questions": [
                    "Relevant question 1?",
                    "Relevant question 2?"
                ],
                "response_hint": "Brief guidance on the desired guest response."
            }},
            "... Additional points as appropriate ..."
        }}
    }},
    "Con": {{
        "description": "Summary and reflection, leaving listeners with a thoughtful question.",
        "script": ["Powerful summary statement.", "Thought-provoking closing question for the audience."]
    }}
}}

Return only valid JSON adhering strictly to this structure without additional explanatory text.
Just return the output in json format without any seperate markers
"""
        response = deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a skilled podcast content planner."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content


In [82]:
outline_generator = PodcastOutlineGenerator (deployment)


In [83]:
# import PodcastOutlineGenerator
topic = "AI in Education"
information = (
    "AI is transforming education through personalized learning, automated grading, intelligent tutoring systems, "
    "and providing accessibility for students with disabilities. Challenges include ethical concerns, data privacy, "
    "and potential biases in algorithms."
)
host_persona = "Alex, an experienced educator"
guest_persona = "Jamie, a technology enthusiast and CEO of a startup"

outline_text = outline_generator.generate_outline(topic, information, host_persona, guest_persona)
print("Generated Outline:\n", outline_text)
# podcast_creator = PodcastCreator(topic, information, host_persona, guest_persona)




Generated Outline:
 {
    "Intro1": {
        "description": "Brief overview of the podcast and general welcome from the host.",
        "script": ["Welcome listeners to today's episode of 'Future of Learning'!", "I'm Alex, your host, and I'm thrilled you're joining us to explore cutting-edge ideas in education."]
    },
    "Intro2": {
        "description": "Introduction of today's topic and brief bio of the guest speaker.",
        "script": ["Today, we dive into a fascinating topic: how Artificial Intelligence is reshaping education.", "Joining us is Jamie, a technology enthusiast and CEO of an innovative startup focusing on AI applications for learning."]
    },
    "Points": {
        "talking_points": {
            "Personalized Learning": {
                "discussion": "Exploring how AI enables tailored educational experiences based on individual student needs.",
                "questions": [
                    "How does AI create personalized learning paths for students?",


In [68]:
class DialogueEnhancer:
    def __init__(self, deployment,model="gpt-4o"):
        self.model = model

    def enhance_section(self, section_name, content, criteria):
        prompt = (
            f"""Given the following podcast section, modify it based on the following criteria: {criteria}

            Section: {section_name}
            Content: {content}

            Provide a compelling and engaging script."""
        )
        response = deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a professional podcast scriptwriter."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content


In [69]:
class SpontaneousDialogueGenerator:
    def __init__(self, deployment, model="gpt-4o"):
        self.model = model

    def generate_dialogue(self, section, section_content, host_persona, guest_persona):
        prompt = (
            f"""Transform the following structured podcast section into a spontaneous dialogue between a host named {host_persona} and a guest named {guest_persona}, naturally including conversational fillers, pauses with possible duration in brackets [pause: 2s], and interaction tags.
            
            Section: {section}
            Content: {section_content}
            
            Include conversational tags like <laughter>, <overlap>, <interruption>, <hesitations> naturally and also include speaker identification like Host:<utt>, Guest: <utt>.
            """
        )
        response = deployment.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert at writing spontaneous and realistic podcast dialogues."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content

In [70]:
dialogue_generator=SpontaneousDialogueGenerator(deployment)
enhancer=DialogueEnhancer(deployment)

In [89]:
outline = json.loads(outline_text)

In [90]:
print(outline)

{'Intro1': {'description': 'Brief overview of the podcast and general welcome from the host.', 'script': ["Welcome listeners to today's episode of 'Future of Learning'!", "I'm Alex, your host, and I'm thrilled you're joining us to explore cutting-edge ideas in education."]}, 'Intro2': {'description': "Introduction of today's topic and brief bio of the guest speaker.", 'script': ['Today, we dive into a fascinating topic: how Artificial Intelligence is reshaping education.', 'Joining us is Jamie, a technology enthusiast and CEO of an innovative startup focusing on AI applications for learning.']}, 'Points': {'talking_points': {'Personalized Learning': {'discussion': 'Exploring how AI enables tailored educational experiences based on individual student needs.', 'questions': ['How does AI create personalized learning paths for students?', 'What are the examples of tools or systems doing this successfully?'], 'response_hint': 'Discuss specific AI tools, algorithms, and how they adapt conten

In [93]:

podcast_dialogues = []
# Introductions
for key in ["Intro1", "Intro2"]:
    
    enhanced_intro = enhancer.enhance_section(
        key,
        " ".join(outline[key]["script"]),
        criteria="Engaging, energetic, welcoming, and enthusiastic"
    )
    podcast_dialogues.append({"section": key, "dialogue": enhanced_intro})

# Main Discussion
tp_key = "Points"
talking_points = outline[tp_key]["talking_points"]
for tp_title, tp_content in talking_points.items():
    section_content = f"{tp_title}: {tp_content['discussion']} " + \
                      " ".join([f"Q: {q}" for q in tp_content["questions"]])
    spontaneous_dialogue = dialogue_generator.generate_dialogue(
        tp_title, section_content, host_persona, guest_persona
    )
    podcast_dialogues.append({"section": tp_title, "dialogue": spontaneous_dialogue})
# Enhance conclusion
conclusion_key = "Con"
enhanced_conclusion = enhancer.enhance_section(
    conclusion_key,
    " ".join(outline[conclusion_key]["script"]),
    criteria="Powerful, thoughtful, reflective, and thought-provoking"
)
podcast_dialogues.append({"section": conclusion_key, "dialogue": enhanced_conclusion})


In [94]:
podcast_dialogues

[{'section': 'Intro1',
  'dialogue': '**Podcast Section: Intro1**\n\n[Upbeat background music playing]\n\nAlex (with energy and excitement): Hello, hello, brilliant minds, and welcome to *Future of Learning*! I’m Alex, your host, and I couldn’t be more thrilled to have you here with me today. Whether you’re driving, taking a coffee break, or just settling in for some inspiration, you’ve landed in the perfect spot to spark your curiosity. \n\nGet ready as we dive into the bold, game-changing ideas shaping the future of education — the tools, the trends, and the mindsets redefining how we learn and grow in this fast-evolving world. So buckle up and let’s explore what’s possible together!\n\n[Pause for music transition] \n\nAlex (enthusiastically): Ready to jump in? Let’s get started!'},
 {'section': 'Intro2',
  'dialogue': '**Podcast Intro2 - Enhanced Script**  \n\n[Upbeat background music fades in]  \n\n**Host:** "Hello and welcome, everyone, to another exciting episode of [Podcast Name

In [95]:
def modify_dialogues_by_persona(dialogues, persona_criteria):
    modified_dialogues = []
    all_dialogues=[]
    for dialogue in dialogues:
        all_dialogues.extend(dialogue['dialogue'])
        
    modified_dialogues = enhancer.enhance_section(
        "Overall Podcast",
        all_dialogues,
        criteria=persona_criteria
    )
    return modified_dialogues

def save_podcast_to_json(self, podcast, filename="podcast.json"):
    with open(filename, 'w') as f:
        json.dump(podcast, f, indent=4)

In [96]:
modified_podcast = modify_dialogues_by_persona(podcast_dialogues, "Conversational, friendly, and engaging")


In [97]:
modified_podcast

"### **Podcast Title: The Future of Learning with Alex**\n\n**[Upbeat background music fades in]**\n\n**Alex (with energy and excitement):**  \nHello, hello, brilliant minds! Welcome to *The Future of Learning*, where we explore ideas shaping the way we learn and grow in this fast-evolving world. I’m Alex, your host, and let me tell you—today’s episode is going to spark your curiosity and electrify your thoughts!  \n\nWhether you’re driving, enjoying a coffee break, or cozied up while seeking inspiration, you’re in the right place. Together, we’ll explore game-changing ideas defining education’s future—tools, trends, and mindsets transforming how we learn. So buckle up—this is going to be a ride like no other!  \n\n**[Pause for music transition]**\n\n**Alex (enthusiastically):**  \nReady to jump in? Let’s get started!  \n\n---\n\n### **Podcast Dialogue: Personalized Learning with AI**\n\n**Alex:**  \nAlright, Jamie, let’s kick off by talking about something that’s been changing the gam